# 📡 LAB 3 Phase 1: RTL-SDR I/Q Data Acquisition via rtl_tcp

## 🎯 วัตถุประสงค์
- เชื่อมต่อกับ RTL-SDR บน Raspberry Pi ผ่าน rtl_tcp
- รับ I/Q samples จาก DAB+ signal (185.360 MHz)
- วิเคราะห์สเปกตรัมและบันทึกข้อมูล
- เปรียบเทียบคุณภาพสัญญาณระหว่าง local และ network

## 📋 ข้อกำหนดเบื้องต้น
- Raspberry Pi 4 รัน `rtl_tcp` server (port 1234)
- FRP tunnel เชื่อมต่อ RPI กับ Colab
- RTL-SDR V4 dongle ต่อกับ RPI

## ⏱️ ระยะเวลา: 45-60 นาที

---
## 📦 ส่วนที่ 1: ติดตั้ง Dependencies

In [ ]:
# ติดตั้ง Python packages
!pip install numpy scipy matplotlib -q

print("✅ Dependencies installed successfully!")

---
## 🔧 ส่วนที่ 2: Import Libraries และตั้งค่า

In [ ]:
import socket
import struct
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

print("📚 Libraries imported")
print(f"🕐 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 🌐 ส่วนที่ 3: ตั้งค่าการเชื่อมต่อ FRP

### ⚠️ TODO: เปลี่ยนค่าตามการตั้งค่าของคุณ
- `FRP_SERVER`: IP Address ของ FRP Server
- `FRP_PORT`: Remote port ที่คุณใช้ (60XX)
- `DAB_FREQUENCY`: ความถี่ DAB+ ในประเทศไทย

In [ ]:
# ===== ตั้งค่าที่นี่ =====
FRP_SERVER = "xxx.xxx.xxx.xxx"  # TODO: ใส่ IP ของ FRP Server
FRP_PORT = 60XX                  # TODO: ใส่ remote port ของคุณ (เช่น 6001)

# พารามิเตอร์ DAB+
DAB_FREQUENCY = 185.360e6  # 185.360 MHz (Bangkok/Phuket)
SAMPLE_RATE = 2.048e6      # 2.048 MHz (DAB+ standard)
GAIN_DB = 30               # Gain 30 dB (ปรับได้)

print(f"📡 FRP Server: {FRP_SERVER}:{FRP_PORT}")
print(f"📻 DAB+ Frequency: {DAB_FREQUENCY/1e6:.3f} MHz")
print(f"📊 Sample Rate: {SAMPLE_RATE/1e6:.3f} Msps")
print(f"📈 Gain: {GAIN_DB} dB")

---
## 🔌 ส่วนที่ 4: ทดสอบการเชื่อมต่อ FRP

ก่อนที่จะเริ่มรับข้อมูล ต้องแน่ใจว่า FRP tunnel ทำงานอยู่

In [ ]:
def test_frp_connection(server, port, timeout=5):
    """
    ทดสอบการเชื่อมต่อ FRP tunnel
    
    TODO: เขียนโค้ดเพื่อ:
    - สร้าง TCP socket
    - ลองเชื่อมต่อไปยัง FRP Server
    - แสดงผลการเชื่อมต่อ
    """
    try:
        # TODO: สร้าง socket
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        
        # TODO: ลองเชื่อมต่อ
        result = sock.connect_ex((server, port))
        
        if result == 0:
            print(f"✅ Connected to {server}:{port}")
            sock.close()
            return True
        else:
            print(f"❌ Cannot connect to {server}:{port}")
            return False
            
    except Exception as e:
        print(f"❌ Connection error: {e}")
        return False

# ทดสอบการเชื่อมต่อ
print("🔍 Testing FRP connection...")
if test_frp_connection(FRP_SERVER, FRP_PORT):
    print("\n🎉 Ready to proceed!")
else:
    print("\n⚠️ Please check:")
    print("   1. FRP Client running on RPI")
    print("   2. rtl_tcp server running on RPI")
    print("   3. Correct FRP_SERVER and FRP_PORT")

---
## 🎛️ ส่วนที่ 5: RTLTCPClient Class

Class สำหรับเชื่อมต่อและควบคุม RTL-SDR ผ่าน rtl_tcp protocol

In [ ]:
class RTLTCPClient:
    """
    Client สำหรับเชื่อมต่อกับ rtl_tcp server
    
    rtl_tcp protocol commands:
    - 0x01: SET_FREQUENCY (4 bytes)
    - 0x02: SET_SAMPLE_RATE (4 bytes)
    - 0x03: SET_GAIN_MODE (4 bytes: 0=auto, 1=manual)
    - 0x04: SET_GAIN (4 bytes: gain in 0.1 dB)
    """
    
    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port
        self.sock = None
        self.connected = False
    
    def connect(self):
        """
        เชื่อมต่อกับ rtl_tcp server
        
        TODO: เขียนโค้ดเพื่อ:
        - สร้าง TCP socket
        - เชื่อมต่อกับ server
        - อ่าน dongle info (12 bytes แรก)
        """
        try:
            # TODO: สร้าง socket
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(10)
            
            # TODO: เชื่อมต่อ
            print(f"🔌 Connecting to {self.hostname}:{self.port}...")
            self.sock.connect((self.hostname, self.port))
            
            # TODO: อ่าน dongle info (12 bytes)
            info = self.sock.recv(12)
            print(f"📡 Dongle info received: {len(info)} bytes")
            
            self.connected = True
            print("✅ Connected to rtl_tcp server")
            return True
            
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
    
    def send_command(self, cmd, param):
        """
        ส่งคำสั่งไปยัง rtl_tcp server
        
        TODO: เขียนโค้ดเพื่อ:
        - Pack command และ parameter เป็น binary
        - ส่งผ่าน socket
        
        Format: >BI = Big-endian Byte + Integer (5 bytes total)
        """
        if not self.connected:
            return False
        
        try:
            # TODO: Pack command (1 byte) + parameter (4 bytes)
            data = struct.pack('>BI', cmd, param)
            
            # TODO: ส่งข้อมูล
            self.sock.send(data)
            return True
            
        except Exception as e:
            print(f"❌ Send command failed: {e}")
            return False
    
    def set_frequency(self, freq_hz):
        """
        ตั้งค่าความถี่
        
        TODO: ส่งคำสั่ง 0x01 + ความถี่ (Hz)
        """
        if self.send_command(0x01, int(freq_hz)):
            print(f"📡 Set frequency: {freq_hz/1e6:.3f} MHz")
            return True
        return False
    
    def set_sample_rate(self, rate_hz):
        """
        ตั้งค่า sample rate
        
        TODO: ส่งคำสั่ง 0x02 + sample rate (Hz)
        """
        if self.send_command(0x02, int(rate_hz)):
            print(f"📊 Set sample rate: {rate_hz/1e6:.3f} Msps")
            return True
        return False
    
    def set_gain_mode(self, manual=True):
        """
        ตั้งค่า gain mode
        
        TODO: ส่งคำสั่ง 0x03 + mode (0=auto, 1=manual)
        """
        mode = 1 if manual else 0
        if self.send_command(0x03, mode):
            print(f"🎚️ Set gain mode: {'manual' if manual else 'auto'}")
            return True
        return False
    
    def set_gain(self, gain_db):
        """
        ตั้งค่า gain (หน่วย dB)
        
        TODO: ส่งคำสั่ง 0x04 + gain (0.1 dB units)
        """
        gain_tenth_db = int(gain_db * 10)
        if self.send_command(0x04, gain_tenth_db):
            print(f"📈 Set gain: {gain_db} dB")
            return True
        return False
    
    def read_samples(self, num_samples):
        """
        อ่าน I/Q samples จาก rtl_tcp
        
        TODO: เขียนโค้ดเพื่อ:
        - คำนวณจำนวน bytes (2 bytes per sample: I + Q)
        - รับข้อมูลจาก socket
        - แปลง uint8 → float32 → complex
        
        Data format: IQIQIQ... (uint8, centered at 127.5)
        """
        if not self.connected:
            return None
        
        try:
            # TODO: คำนวณจำนวน bytes
            num_bytes = num_samples * 2  # 2 bytes per complex sample
            data = b''
            
            # TODO: รับข้อมูลจาก socket
            print(f"📥 Reading {num_samples} samples ({num_bytes} bytes)...")
            while len(data) < num_bytes:
                chunk_size = min(8192, num_bytes - len(data))
                chunk = self.sock.recv(chunk_size)
                if not chunk:
                    break
                data += chunk
            
            print(f"✅ Received {len(data)} bytes")
            
            # TODO: แปลง bytes → numpy array (uint8)
            iq_uint8 = np.frombuffer(data, dtype=np.uint8)
            
            # TODO: แปลง uint8 → float (-1.0 to +1.0)
            iq_float = (iq_uint8 - 127.5) / 127.5
            
            # TODO: แยก I และ Q
            i_samples = iq_float[0::2]  # ตัวคู่
            q_samples = iq_float[1::2]  # ตัวคี่
            
            # TODO: สร้าง complex samples
            samples = i_samples + 1j * q_samples
            
            return samples
            
        except Exception as e:
            print(f"❌ Read samples failed: {e}")
            return None
    
    def close(self):
        """
        ปิดการเชื่อมต่อ
        
        TODO: ปิด socket connection
        """
        if self.sock:
            self.sock.close()
            self.connected = False
            print("🔌 Connection closed")

print("✅ RTLTCPClient class defined")

---
## 🚀 ส่วนที่ 6: เชื่อมต่อและตั้งค่า RTL-SDR

In [ ]:
# TODO: สร้าง RTLTCPClient instance
print("🔧 Creating RTL-TCP client...")
sdr = RTLTCPClient(hostname=FRP_SERVER, port=FRP_PORT)

# TODO: เชื่อมต่อกับ rtl_tcp server
if sdr.connect():
    print("\n⚙️ Configuring RTL-SDR parameters...")
    
    # TODO: ตั้งค่า sample rate
    sdr.set_sample_rate(SAMPLE_RATE)
    
    # TODO: ตั้งค่าความถี่
    sdr.set_frequency(DAB_FREQUENCY)
    
    # TODO: ตั้งค่า gain mode เป็น manual
    sdr.set_gain_mode(manual=True)
    
    # TODO: ตั้งค่า gain
    sdr.set_gain(GAIN_DB)
    
    print("\n✅ RTL-SDR configured successfully!")
    print(f"📡 Ready to capture DAB+ signal at {DAB_FREQUENCY/1e6:.3f} MHz")
else:
    print("\n❌ Failed to connect to RTL-SDR")

---
## 📊 ส่วนที่ 7: รับและวิเคราะห์ I/Q Samples

รับ I/Q samples และวิเคราะห์สเปกตรัม

In [ ]:
# TODO: กำหนดจำนวน samples ที่ต้องการ
NUM_SAMPLES = 256 * 1024  # 256K samples (~0.125 วินาที ที่ 2.048 Msps)

print(f"📡 Capturing {NUM_SAMPLES} samples...")
print(f"⏱️ Duration: ~{NUM_SAMPLES/SAMPLE_RATE:.3f} seconds")

# TODO: รับ samples
start_time = time.time()
samples = sdr.read_samples(NUM_SAMPLES)
capture_time = time.time() - start_time

if samples is not None:
    print(f"\n✅ Capture completed in {capture_time:.2f} seconds")
    print(f"📊 Samples shape: {samples.shape}")
    print(f"📊 Data type: {samples.dtype}")
    
    # TODO: คำนวณ signal statistics
    signal_power = np.mean(np.abs(samples)**2)
    signal_rms = np.sqrt(signal_power)
    max_amplitude = np.max(np.abs(samples))
    
    print(f"\n📈 Signal Statistics:")
    print(f"   Power: {signal_power:.6f}")
    print(f"   RMS: {signal_rms:.6f}")
    print(f"   Max amplitude: {max_amplitude:.6f}")
    
    # TODO: บันทึกเป็นไฟล์ binary (optional)
    filename = f"iq_samples_{datetime.now().strftime('%Y%m%d_%H%M%S')}.npy"
    np.save(filename, samples)
    print(f"\n💾 Saved to: {filename}")
else:
    print("\n❌ Failed to capture samples")

---
## 📈 ส่วนที่ 8: วิเคราะห์และแสดงผลสเปกตรัม

In [ ]:
if samples is not None:
    # TODO: สร้าง figure สำหรับแสดงผล
    fig, axes = plt.subplots(3, 1, figsize=(15, 10))
    
    # ============ Plot 1: Time Domain ============
    ax1 = axes[0]
    num_display = min(1000, len(samples))  # แสดง 1000 samples แรก
    
    # TODO: Plot I และ Q samples
    ax1.plot(np.real(samples[:num_display]), label='I (In-phase)', alpha=0.7, linewidth=0.8)
    ax1.plot(np.imag(samples[:num_display]), label='Q (Quadrature)', alpha=0.7, linewidth=0.8)
    ax1.set_xlabel('Sample')
    ax1.set_ylabel('Amplitude')
    ax1.set_title(f'Time Domain - I/Q Samples (First {num_display} samples)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # ============ Plot 2: Frequency Spectrum (FFT) ============
    ax2 = axes[1]
    
    # TODO: คำนวณ FFT
    fft_data = np.fft.fftshift(np.fft.fft(samples))
    fft_db = 20 * np.log10(np.abs(fft_data) + 1e-10)
    
    # TODO: คำนวณ frequency bins
    freqs = np.fft.fftshift(np.fft.fftfreq(len(samples), 1/SAMPLE_RATE))
    
    # TODO: Plot spectrum
    ax2.plot(freqs/1e6, fft_db, linewidth=0.8)
    ax2.set_xlabel('Frequency Offset (MHz)')
    ax2.set_ylabel('Power (dB)')
    ax2.set_title(f'Frequency Spectrum at {DAB_FREQUENCY/1e6:.3f} MHz')
    ax2.grid(True, alpha=0.3)
    
    # TODO: หา peak frequency
    peak_idx = np.argmax(fft_db)
    peak_freq = freqs[peak_idx]/1e6
    peak_power = fft_db[peak_idx]
    
    ax2.axvline(peak_freq, color='red', linestyle='--', alpha=0.5, linewidth=1, 
                label=f'Peak: {peak_freq:.3f} MHz @ {peak_power:.1f} dB')
    ax2.legend()
    
    # ============ Plot 3: Power Spectral Density ============
    ax3 = axes[2]
    
    # TODO: คำนวณ PSD
    from scipy import signal as scipy_signal
    f_psd, psd = scipy_signal.welch(samples, fs=SAMPLE_RATE, nperseg=2048)
    psd_db = 10 * np.log10(psd + 1e-10)
    
    # TODO: Plot PSD
    ax3.plot(f_psd/1e6, psd_db, linewidth=0.8)
    ax3.set_xlabel('Frequency Offset (MHz)')
    ax3.set_ylabel('PSD (dB/Hz)')
    ax3.set_title('Power Spectral Density (Welch Method)')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Spectrum Analysis:")
    print(f"   Peak frequency offset: {peak_freq:.3f} MHz")
    print(f"   Peak power: {peak_power:.1f} dB")
    print(f"   Absolute frequency: {DAB_FREQUENCY/1e6 + peak_freq:.3f} MHz")

---
## 🔍 ส่วนที่ 9: วิเคราะห์คุณภาพสัญญาณ

In [ ]:
if samples is not None:
    # TODO: คำนวณ SNR (Signal-to-Noise Ratio) เบื้องต้น
    # แยก signal (bandwidth ±0.8 MHz) จาก noise
    
    signal_bandwidth = 1.536e6  # DAB+ bandwidth ~1.536 MHz
    signal_bins = int(signal_bandwidth / (SAMPLE_RATE / len(samples)))
    
    center_idx = len(fft_db) // 2
    signal_range = slice(center_idx - signal_bins//2, center_idx + signal_bins//2)
    
    signal_power_db = np.mean(fft_db[signal_range])
    
    # Noise: ใช้ส่วนข้างนอก signal bandwidth
    noise_indices = np.concatenate([
        np.arange(0, center_idx - signal_bins//2),
        np.arange(center_idx + signal_bins//2, len(fft_db))
    ])
    noise_power_db = np.mean(fft_db[noise_indices])
    
    snr_estimate = signal_power_db - noise_power_db
    
    print(f"\n📡 Signal Quality Estimation:")
    print(f"   Signal power: {signal_power_db:.1f} dB")
    print(f"   Noise power: {noise_power_db:.1f} dB")
    print(f"   SNR (estimated): {snr_estimate:.1f} dB")
    
    if snr_estimate > 15:
        print(f"   ✅ Signal quality: Excellent (SNR > 15 dB)")
    elif snr_estimate > 10:
        print(f"   ✅ Signal quality: Good (SNR > 10 dB)")
    elif snr_estimate > 5:
        print(f"   ⚠️ Signal quality: Fair (SNR > 5 dB)")
    else:
        print(f"   ❌ Signal quality: Poor (SNR < 5 dB)")
    
    # TODO: แสดง constellation diagram (IQ plot)
    plt.figure(figsize=(8, 8))
    decimation = max(1, len(samples) // 10000)  # แสดงแค่ 10000 จุด
    plt.scatter(np.real(samples[::decimation]), 
                np.imag(samples[::decimation]), 
                alpha=0.1, s=1)
    plt.xlabel('I (In-phase)')
    plt.ylabel('Q (Quadrature)')
    plt.title('Constellation Diagram (I/Q Scatter Plot)')
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.show()

---
## 🔄 ส่วนที่ 10: Real-time Monitoring (Optional)

รับและแสดงผลสเปกตรัมแบบ real-time ทุกๆ 3 วินาที

In [ ]:
# TODO: Real-time monitoring loop
from IPython import display

UPDATE_INTERVAL = 3  # seconds
NUM_ITERATIONS = 5   # จำนวนรอบที่จะแสดงผล

print(f"📡 Starting real-time monitoring...")
print(f"🔄 Update every {UPDATE_INTERVAL} seconds")
print(f"📊 Total iterations: {NUM_ITERATIONS}")
print(f"⏹️ Stop: Interrupt the cell\n")

try:
    for iteration in range(NUM_ITERATIONS):
        start_time = time.time()
        
        print(f"\n📡 Iteration {iteration + 1}/{NUM_ITERATIONS} - {datetime.now().strftime('%H:%M:%S')}")
        
        # TODO: รับ samples
        samples = sdr.read_samples(NUM_SAMPLES)
        
        if samples is not None:
            # TODO: คำนวณ FFT
            fft_data = np.fft.fftshift(np.fft.fft(samples))
            fft_db = 20 * np.log10(np.abs(fft_data) + 1e-10)
            freqs = np.fft.fftshift(np.fft.fftfreq(len(samples), 1/SAMPLE_RATE))
            
            # TODO: Clear และสร้าง plot ใหม่
            display.clear_output(wait=True)
            
            plt.figure(figsize=(15, 5))
            plt.plot(freqs/1e6, fft_db, linewidth=0.8)
            plt.xlabel('Frequency Offset (MHz)')
            plt.ylabel('Power (dB)')
            plt.title(f'Real-time Spectrum - {datetime.now().strftime("%H:%M:%S")}')
            plt.grid(True, alpha=0.3)
            plt.show()
            
            # TODO: แสดง statistics
            signal_power = np.mean(np.abs(samples)**2)
            print(f"   Signal power: {signal_power:.6f}")
            print(f"   Peak: {np.max(fft_db):.1f} dB")
        
        # TODO: รอให้ครบ UPDATE_INTERVAL
        elapsed = time.time() - start_time
        wait_time = UPDATE_INTERVAL - elapsed
        if wait_time > 0:
            print(f"   ⏱️ Processing time: {elapsed:.2f}s, waiting {wait_time:.1f}s...")
            time.sleep(wait_time)

except KeyboardInterrupt:
    print("\n⏹️ Monitoring stopped by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Real-time monitoring completed")

---
## 🧹 ส่วนที่ 11: Cleanup และปิดการเชื่อมต่อ

In [ ]:
# TODO: ปิดการเชื่อมต่อ
print("🧹 Cleaning up...")
sdr.close()
print("\n✅ Lab 3 Phase 1 completed!")
print(f"📁 Saved files: {filename}")

---
## 📝 สรุป Lab 3 Phase 1

### ✅ สิ่งที่ทำสำเร็จ:
1. ✅ เชื่อมต่อกับ RTL-SDR ผ่าน rtl_tcp และ FRP tunnel
2. ✅ ตั้งค่าความถี่ DAB+ Thailand (185.360 MHz)
3. ✅ รับและบันทึก I/Q samples
4. ✅ วิเคราะห์สเปกตรัมด้วย FFT
5. ✅ ประเมินคุณภาพสัญญาณ (SNR)
6. ✅ แสดงผล constellation diagram
7. ✅ Real-time spectrum monitoring

### 🎯 ทักษะที่ได้รับ:
- การเชื่อมต่อ RTL-SDR ผ่าน network (rtl_tcp protocol)
- การทำงานกับ I/Q data และ complex samples
- การวิเคราะห์สัญญาณด้วย FFT และ PSD
- การประเมินคุณภาพสัญญาณ RF
- Data visualization ด้วย matplotlib

### 🔜 ขั้นต่อไป: Lab 3 Phase 2
- แปลง I/Q data เป็น ETI stream ด้วย eti-cmdline
- ติดตามสถานะ sync และ error rate
- วิเคราะห์ ETI frame structure

---

## 📚 เอกสารอ้างอิง
- [RTL-SDR Blog](https://www.rtl-sdr.com/)
- [rtl_tcp Protocol](https://osmocom.org/projects/rtl-sdr/wiki/Rtl-sdr)
- [DAB+ Standard ETSI EN 300 401](https://www.etsi.org/deliver/etsi_en/300400_300499/300401/02.01.01_60/en_300401v020101p.pdf)
- [WorldDAB Thailand](https://www.worlddab.org/countries/thailand)